In [5]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from textblob import TextBlob

In [6]:
df = pd.read_csv("../../amazon_reviews_us_Grocery_v1_00.tsv", sep='\t', quoting=3)
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix,...",Grocery,5,0,0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nu...",2015-08-31
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0,0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and ...",2015-08-31
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Ma...,Grocery,5,0,0,N,N,Five Stars,This green tea tastes so good! My girlfriend l...,2015-08-31
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0,0,N,Y,Five Stars,I love Melissa's brand but this is a great sec...,2015-08-31
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-...",Grocery,5,0,0,N,Y,Five Stars,good,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402453,US,50385921,RZVFIWM72OSA0,B00004TBB0,738532654,Celestial Seasonings - Herbal Tea Caffeine Fre...,Grocery,4,0,4,N,N,Different,The Amor Belhom Duo are likely the only musici...,2000-12-05
2402454,US,50377094,R35K50CF47311S,B00000K1X9,161697639,479 NATL SHOW HRSE BAY PINTO 6,Grocery,5,0,0,N,N,This Horse Is Gorgourges!,Being a Breyer collecter almost all of ...,1999-12-29
2402455,US,50740327,R1N80O7XL5ODNC,B00000K1X9,161697639,479 NATL SHOW HRSE BAY PINTO 6,Grocery,5,5,5,N,N,"brilliant, realistic model!",This model is beautiful. each model is hand p...,1999-11-21
2402456,US,51700314,R1E9RSEX4R6TIR,B00000K1X9,161697639,479 NATL SHOW HRSE BAY PINTO 6,Grocery,5,1,1,N,N,don't have it yet..but will soon,As a person who has collected Breyers all my l...,1999-11-17


In [7]:
df = df.dropna()

In [ ]:
polarity = []
subjectivity = []
for line in df['review_body']:
    line = line.replace('<br />', '')
    blob = TextBlob(line)
    pol = []
    sub = []
    for sentence in blob.sentences:
        pol.append(sentence.sentiment.polarity)
        sub.append(sentence.sentiment.subjectivity)
    polarity.append(np.array(pol).mean())
    subjectivity.append(np.array(sub).mean())
df['polarity'] = polarity
df['subjectivity'] = subjectivity

In [ ]:
df.loc[(df['polarity'] < 0) & (df['subjectivity'] < 0.1)]['review_body']

In [44]:
df.to_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv',index=False)

In [12]:
df.query('review_body.str.contains("sick", na=False) & star_rating == 1', engine='python')

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
2709,US,11848670,RB8NX7A6OH4GQ,B007D7OUE0,490870028,TruffleHunter White Truffle Oil (3.38 Oz),Grocery,1,5,6,N,Y,ALL Truffle Oils Are Made From Chemicals,This tastes good but I got extremely sick both...,2015-08-31
2980,US,36498655,R2BTW6A20GO7PV,B00ES89PWY,449397395,Good Earth Organic Tea,Grocery,1,0,0,N,N,Aggressively bad.,From the moment the first sip touches the tip ...,2015-08-31
3212,US,50782674,R34QOM3I2B60FS,B00CS5T2C2,983699690,Spectrum Essentials Chia and Flax Seed Decaden...,Grocery,1,2,2,N,N,Tastes like soap!,"I dont know what I was expecting, but I put so...",2015-08-31
3679,US,42665845,RQ0AOG7A18K8P,B004L5UCQM,512568416,"Fritos Bean Dip, 3.125 Oz Can (Pack of 24)",Grocery,1,0,1,N,N,Too Stinkin' HOT!,TOO HOT! They finally ruined it!<br /><br />Ca...,2015-08-30
3827,US,39311033,R2WAXJV0PFJMU7,B008AS1724,493037893,"Hale Tea Black Tea, Decaf Vanilla Red Berries,...",Grocery,1,1,1,N,Y,NOT FULLY DECAF,I bought this assuming it was decaf. I brewed...,2015-08-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396689,US,50891467,R1SZF8NDQ7SDVR,B000EMRHWI,934807395,"Just the Cheese Snack Bars, 0.5-Ounce Bars (Pa...",Grocery,1,8,11,N,N,Absoutely disgusting.,I read the product description and thought to ...,2006-07-20
2398717,US,51047759,R240KUO9AKIYO8,B000249FYG,298297803,Mariza Durian Jam,Grocery,1,2,2,N,N,Oh MY GOD!,"The durian jelly tastes sweet at first, then i...",2006-01-31
2398720,US,51047759,RSDPJDCAKASZ9,B000BKGYMM,786662870,Garden - Durian Wafer Cookies (7.0 Oz),Grocery,1,2,3,N,N,Not for the faint of heart,The durian wafer cookie looks very much like v...,2006-01-31
2400767,US,52195402,REFE39RI473W7,B0002BTD14,747324490,1 Case of Candy Cigarettes,Grocery,1,0,0,N,N,WARNING - Product encourages role modeling dea...,If you MUST give a child a candy containing su...,2005-02-19


In [13]:
df.query('review_body.str.contains("moldy", na=False) & star_rating == 1', engine='python')

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
301,US,15134590,R2OK571OMZ0ZDK,B004FEGLO6,359679189,"Mott's 100% Original Apple Juice, 6.75 fl oz b...",Grocery,1,2,2,N,Y,I had purchased this product before and been v...,I had to throw away one 8 pack of this juice b...,2015-08-31
10280,US,11255558,R2K4A419K1CXMM,B00L9O05OM,791342115,Italian Dried Porcini Mushrooms 1 lb (16 oz) B...,Grocery,1,0,0,N,Y,Nasty,Mushrooms are moldy!,2015-08-28
13371,US,49224849,R34JVBHYSK0HFT,B00WJ1DORO,521501205,5lbs Organic Einkorn Berries (Farro Piccolo),Grocery,1,2,2,N,Y,Einkorn not for sprouting.....,"ok, I am not sure if I just rec'd a bad bag or...",2015-08-27
16068,US,31200422,R3ECYPOGI0YQD2,B005HG9ERW,192623861,Essentia 9.5 pH Drinking Water,Grocery,1,11,11,N,Y,Mold and/or something gross floating on most o...,Nasty floating moldy stuff in this water.....d...,2015-08-26
22405,US,38751059,R9VPXX6BP0GWR,B007K689MI,971794757,Old Wisconsin Snack Sticks,Grocery,1,0,0,N,Y,Not great shelf life. Gobble bummer.,Ordered July 5th. Just threw away opened (but ...,2015-08-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384714,US,36488846,RH6HZHFVYKFCS,B000EUJS1S,377307874,"Sorbee No Sugar Added Strawberry Ceral Bars, S...",Grocery,1,0,0,N,N,All Moldy,All the strawberry bars were moldy. I wish I h...,2007-03-02
2387452,US,33323251,R1FLQJCWJ74P0P,B000J2JW6W,510699755,Holiday Sale - 2.2 Pounds of Chocolate Truffle...,Grocery,1,11,15,N,N,Moldy Truffles,Last night I spoke with Jane Newman and she to...,2007-01-18
2387600,US,52805315,R16VBCTB97C5B5,B000EZOOZI,958601583,SnackMasters Range Grown Turkey Jerky,Grocery,1,2,4,N,N,It was moldy!,I ordered an 8 pack of this jerkey to try and ...,2007-01-16
2395559,US,16701186,R2WK1469U9XZOZ,B00032G1S0,753469671,"Tuscan Dairy Whole Vitamin D Milk, Gallon, 128 oz",Grocery,1,12,25,N,N,so i got this milk,"i got this milk, it was rancid and lumpy... so...",2006-08-09
